In [15]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import psycopg2

def load_website(url,prefix):
    try:
        response = requests.get(prefix+url)
        return BeautifulSoup(response.text)
    except Exception as err:
        print(f'ERROR: {err}')

In [16]:
def scrape_and_insert(cat, j, articles, k, cur, conn, tablename):
    #Works for scraping beautiful soup product item from tiki product page
    try:
        #scrape and assign to variables
        images = articles[k].img['src']
        fprice = int(articles[k].find_all("span",{"class":"final-price"})[0].text.strip().split()[0].strip('đ').replace('.',''))
        rprice = [0 if articles[k].find_all("span",{"class":"price-regular"})[0].text == '' else int(articles[k].find_all("span",{"class":"price-regular"})[0].text.strip('đ').replace('.',''))][0]
        discount = [0 if len(articles[k].find_all("span",{"class":"final-price"})[0].text.strip().split()) == 1 else int(articles[k].find_all("span",{"class":"final-price"})[0].text.strip().split()[1].split('%')[0])][0]
        seller = articles[k]['data-brand'].replace('\'','').replace('"','')
        titles = articles[k].a['title'].strip().replace('\'','').replace('"','')
        subcategory = articles[k]['data-category'].strip().replace('\'','').replace('"','')
        category = cat[j][0]
#         num_reviews = [int(articles[k].find_all('p',{"class":'review'})[0].text.strip('\(\)').split()[0]) if articles[k].find_all('p',{"class":'review'}) != [] or articles[k].find_all('p',{"class":'review'}) != 'Chưa có nhận xét' else 0][0]
        if articles[k].find_all('p',{"class":'review'}) == [] or articles[k].find_all('p',{"class":'review'})[0].text == 'Chưa có nhận xét':
            num_reviews = 0
        else:
            num_reviews = int(articles[k].find_all('p',{"class":'review'})[0].text.strip('\(\)').split()[0])

        ratings = [int(articles[k].find_all('span',{"class":'rating-content'})[0].find('span')['style'].split(':')[1].split('%')[0]) if articles[k].find_all('span',{"class":'rating-content'}) != [] else 0][0]
        tikinow = [0 if articles[k].find_all('i',{"class":"tikicon icon-tikinow-20"}) == [] else 1][0]
        productlink = articles[k].a['href']
        #build query string
        query = f"""INSERT INTO {tablename}(images, fprice, category, subcategory, titles, seller, rprice, discount, ratings, num_reviews, tikinow, productlink)
                    VALUES('{images}','{fprice}', '{category}', '{subcategory}','{titles}','{seller}','{rprice}','{discount}','{ratings}','{num_reviews}','{tikinow}','{productlink}');"""
        #commit to connection
        cur.execute(query)
        conn.commit()
    except Exception as err:
        print(err, k)

In [18]:
#load tiki home page
soup = load_website('https://tiki.vn/',prefix='')
#scrape the categories and their links and store in array
categories = soup.find_all('a',{"class":'MenuItem__MenuLink-tii3xq-1 efuIbv'})
category, link = [], []
for h in range(len(categories)):
    try:
        link.append(categories[h]['href'])
        category.append(categories[h].text)
    except:
        print('pass')
cat = list(zip(category,link))

In [17]:
#create connect to DB and cursor
conn = psycopg2.connect("dbname=thuctamdb user=postgres password=thuctam")

cur = conn.cursor()

In [10]:
#create new table
tablename = 'test'
query = f'''
CREATE TABLE {tablename}(
   id SERIAL PRIMARY KEY,
   images TEXT,
   fprice INT,
   category VARCHAR(1024),
   subcategory VARCHAR(1024),
   titles VARCHAR(1024),
   seller VARCHAR(255),
   rprice INT,
   discount INT,
   ratings INT,
   num_reviews INT,
   tikinow INT,
   productlink TEXT
);'''
cur.execute(query)
conn.commit()

In [11]:
for j in range(len(cat)):
    try:
        soup = load_website(cat[j][1],prefix='')
        # Read next page cursor at the bottom of a product page
        links = soup.find_all('div',{"class":'list-pager'})
        
        while links[0].find_all('a', {"class": "next"}) != []:
            try:
                soup = load_website(links[0].find_all('a', {"class": "next"})[0]['href'],prefix='https://tiki.vn')
                articles = soup.find_all('div', {"class":"product-item"})
                print('Reading',cat[j][0],links[0].find_all('a', {"class": "next"})[0]['href'].split('&')[1],sep=' ')
                links = soup.find_all('div',{"class":'list-pager'})
                for i in range(len(articles)):
                    scrape_and_insert(cat, j, articles, i, cur, conn, tablename)
                
            except:
                continue
                
        soup = load_website(cat[j][1],prefix='')
        articles = soup.find_all('div', {"class":'product-item'})
        print('Reading '+cat[j][1])
        for k in range(len(articles)):
            scrape_and_insert(cat, j, articles, k, cur, conn, tablename)
        
        #While next page cursor is not empty, read next page cursor to move to next product page
        
    except:
        continue
print("SUCCESS!")

Reading Điện Thoại - Máy Tính Bảng page=2
Reading Điện Thoại - Máy Tính Bảng page=3
Reading Điện Thoại - Máy Tính Bảng page=4
Reading Điện Thoại - Máy Tính Bảng page=5
Reading Điện Thoại - Máy Tính Bảng page=6
Reading Điện Thoại - Máy Tính Bảng page=7
Reading Điện Thoại - Máy Tính Bảng page=8
Reading Điện Thoại - Máy Tính Bảng page=9
Reading Điện Thoại - Máy Tính Bảng page=10
Reading Điện Thoại - Máy Tính Bảng page=11
Reading Điện Thoại - Máy Tính Bảng page=12
Reading Điện Thoại - Máy Tính Bảng page=13
Reading Điện Thoại - Máy Tính Bảng page=14
Reading Điện Thoại - Máy Tính Bảng page=15
Reading Điện Thoại - Máy Tính Bảng page=16
Reading Điện Thoại - Máy Tính Bảng page=17
Reading https://tiki.vn/dien-thoai-may-tinh-bang/c1789?src=c.1789.hamburger_menu_fly_out_banner
Reading Điện Tử - Điện Lạnh page=2
Reading Điện Tử - Điện Lạnh page=3
Reading Điện Tử - Điện Lạnh page=4
Reading Điện Tử - Điện Lạnh page=5
Reading Điện Tử - Điện Lạnh page=6
Reading Điện Tử - Điện Lạnh page=7
Reading Điện T

Reading Phụ Kiện - Thiết Bị Số page=50
Reading Phụ Kiện - Thiết Bị Số page=51
Reading Phụ Kiện - Thiết Bị Số page=52
Reading Phụ Kiện - Thiết Bị Số page=53
Reading Phụ Kiện - Thiết Bị Số page=54
Reading Phụ Kiện - Thiết Bị Số page=55
Reading Phụ Kiện - Thiết Bị Số page=56
Reading Phụ Kiện - Thiết Bị Số page=57
Reading Phụ Kiện - Thiết Bị Số page=58
Reading Phụ Kiện - Thiết Bị Số page=59
Reading Phụ Kiện - Thiết Bị Số page=60
Reading Phụ Kiện - Thiết Bị Số page=61
Reading Phụ Kiện - Thiết Bị Số page=62
Reading Phụ Kiện - Thiết Bị Số page=63
Reading Phụ Kiện - Thiết Bị Số page=64
Reading Phụ Kiện - Thiết Bị Số page=65
Reading Phụ Kiện - Thiết Bị Số page=66
Reading Phụ Kiện - Thiết Bị Số page=67
Reading Phụ Kiện - Thiết Bị Số page=68
Reading Phụ Kiện - Thiết Bị Số page=69
Reading Phụ Kiện - Thiết Bị Số page=70
Reading Phụ Kiện - Thiết Bị Số page=71
Reading Phụ Kiện - Thiết Bị Số page=72
Reading Phụ Kiện - Thiết Bị Số page=73
Reading Phụ Kiện - Thiết Bị Số page=74
Reading Phụ Kiện - Thiết 

Reading Laptop - Thiết bị IT page=53
Reading Laptop - Thiết bị IT page=54
Reading Laptop - Thiết bị IT page=55
Reading Laptop - Thiết bị IT page=56
Reading Laptop - Thiết bị IT page=57
Reading Laptop - Thiết bị IT page=58
Reading Laptop - Thiết bị IT page=59
Reading Laptop - Thiết bị IT page=60
Reading Laptop - Thiết bị IT page=61
Reading Laptop - Thiết bị IT page=62
Reading Laptop - Thiết bị IT page=63
Reading Laptop - Thiết bị IT page=64
Reading Laptop - Thiết bị IT page=65
Reading Laptop - Thiết bị IT page=66
Reading Laptop - Thiết bị IT page=67
Reading Laptop - Thiết bị IT page=68
Reading Laptop - Thiết bị IT page=69
Reading Laptop - Thiết bị IT page=70
Reading Laptop - Thiết bị IT page=71
Reading Laptop - Thiết bị IT page=72
Reading Laptop - Thiết bị IT page=73
Reading Laptop - Thiết bị IT page=74
Reading Laptop - Thiết bị IT page=75
Reading Laptop - Thiết bị IT page=76
Reading Laptop - Thiết bị IT page=77
Reading Laptop - Thiết bị IT page=78
Reading Laptop - Thiết bị IT page=79
R

Reading Máy Ảnh - Quay Phim page=66
Reading Máy Ảnh - Quay Phim page=67
Reading Máy Ảnh - Quay Phim page=68
Reading Máy Ảnh - Quay Phim page=69
Reading Máy Ảnh - Quay Phim page=70
Reading Máy Ảnh - Quay Phim page=71
Reading Máy Ảnh - Quay Phim page=72
Reading Máy Ảnh - Quay Phim page=73
Reading Máy Ảnh - Quay Phim page=74
Reading Máy Ảnh - Quay Phim page=75
Reading Máy Ảnh - Quay Phim page=76
Reading Máy Ảnh - Quay Phim page=77
Reading Máy Ảnh - Quay Phim page=78
Reading Máy Ảnh - Quay Phim page=79
Reading Máy Ảnh - Quay Phim page=80
Reading Máy Ảnh - Quay Phim page=81
Reading Máy Ảnh - Quay Phim page=82
Reading Máy Ảnh - Quay Phim page=83
Reading Máy Ảnh - Quay Phim page=84
Reading Máy Ảnh - Quay Phim page=85
Reading Máy Ảnh - Quay Phim page=86
Reading Máy Ảnh - Quay Phim page=87
Reading Máy Ảnh - Quay Phim page=88
Reading Máy Ảnh - Quay Phim page=89
Reading Máy Ảnh - Quay Phim page=90
Reading Máy Ảnh - Quay Phim page=91
Reading Máy Ảnh - Quay Phim page=92
Reading Máy Ảnh - Quay Phim 

Reading Điện Gia Dụng page=99
Reading Điện Gia Dụng page=100
Reading Điện Gia Dụng page=101
Reading Điện Gia Dụng page=102
Reading Điện Gia Dụng page=103
Reading Điện Gia Dụng page=104
Reading Điện Gia Dụng page=105
Reading Điện Gia Dụng page=106
Reading Điện Gia Dụng page=107
Reading Điện Gia Dụng page=108
Reading Điện Gia Dụng page=109
Reading Điện Gia Dụng page=110
Reading Điện Gia Dụng page=111
Reading Điện Gia Dụng page=112
Reading Điện Gia Dụng page=113
Reading Điện Gia Dụng page=114
Reading Điện Gia Dụng page=115
Reading Điện Gia Dụng page=116
Reading Điện Gia Dụng page=117
Reading Điện Gia Dụng page=118
Reading Điện Gia Dụng page=119
Reading Điện Gia Dụng page=120
Reading Điện Gia Dụng page=121
Reading Điện Gia Dụng page=122
Reading Điện Gia Dụng page=123
Reading Điện Gia Dụng page=124
Reading Điện Gia Dụng page=125
Reading Điện Gia Dụng page=126
Reading Điện Gia Dụng page=127
Reading Điện Gia Dụng page=128
Reading Điện Gia Dụng page=129
Reading Điện Gia Dụng page=130
Reading Đ

...

Reading Thời trang - Phụ kiện page=190
Reading Thời trang - Phụ kiện page=191
Reading Thời trang - Phụ kiện page=192
Reading Thời trang - Phụ kiện page=193
Reading Thời trang - Phụ kiện page=194
Reading Thời trang - Phụ kiện page=195
Reading Thời trang - Phụ kiện page=196
Reading Thời trang - Phụ kiện page=197
Reading Thời trang - Phụ kiện page=198
Reading Thời trang - Phụ kiện page=199
Reading Thời trang - Phụ kiện page=200
Reading Thời trang - Phụ kiện page=201
Reading Thời trang - Phụ kiện page=202
Reading Thời trang - Phụ kiện page=203
Reading Thời trang - Phụ kiện page=204
Reading Thời trang - Phụ kiện page=205
Reading Thời trang - Phụ kiện page=206
Reading Thời trang - Phụ kiện page=207
Reading Thời trang - Phụ kiện page=208
Reading Thời trang - Phụ kiện page=209
Reading Thể Thao - Dã Ngoại page=2
Reading Thể Thao - Dã Ngoại page=3
Reading Thể Thao - Dã Ngoại page=4
Reading Thể Thao - Dã Ngoại page=5
Reading Thể Thao - Dã Ngoại page=6
Reading Thể Thao - Dã Ngoại page=7
Reading Th

Reading Thể Thao - Dã Ngoại page=206
Reading Thể Thao - Dã Ngoại page=207
Reading Thể Thao - Dã Ngoại page=208
Reading Thể Thao - Dã Ngoại page=209
Reading Xe Máy, Ô tô, Xe Đạp page=2
Reading Xe Máy, Ô tô, Xe Đạp page=3
Reading Xe Máy, Ô tô, Xe Đạp page=4
Reading Xe Máy, Ô tô, Xe Đạp page=5
Reading Xe Máy, Ô tô, Xe Đạp page=6
Reading Xe Máy, Ô tô, Xe Đạp page=7
Reading Xe Máy, Ô tô, Xe Đạp page=8
Reading Xe Máy, Ô tô, Xe Đạp page=9
Reading Xe Máy, Ô tô, Xe Đạp page=10
Reading Xe Máy, Ô tô, Xe Đạp page=11
Reading Xe Máy, Ô tô, Xe Đạp page=12
Reading Xe Máy, Ô tô, Xe Đạp page=13
Reading Xe Máy, Ô tô, Xe Đạp page=14
Reading Xe Máy, Ô tô, Xe Đạp page=15
Reading Xe Máy, Ô tô, Xe Đạp page=16
Reading Xe Máy, Ô tô, Xe Đạp page=17
Reading Xe Máy, Ô tô, Xe Đạp page=18
Reading Xe Máy, Ô tô, Xe Đạp page=19
Reading Xe Máy, Ô tô, Xe Đạp page=20
Reading Xe Máy, Ô tô, Xe Đạp page=21
Reading Xe Máy, Ô tô, Xe Đạp page=22
Reading Xe Máy, Ô tô, Xe Đạp page=23
Reading Xe Máy, Ô tô, Xe Đạp page=24
Reading X

Reading Hàng quốc tế page=11
Reading Hàng quốc tế page=12
Reading Hàng quốc tế page=13
Reading Hàng quốc tế page=14
Reading Hàng quốc tế page=15
Reading Hàng quốc tế page=16
Reading Hàng quốc tế page=17
Reading Hàng quốc tế page=18
Reading Hàng quốc tế page=19
Reading Hàng quốc tế page=20
Reading Hàng quốc tế page=21
Reading Hàng quốc tế page=22
Reading Hàng quốc tế page=23
Reading Hàng quốc tế page=24
Reading Hàng quốc tế page=25
Reading Hàng quốc tế page=26
Reading Hàng quốc tế page=27
Reading Hàng quốc tế page=28
Reading Hàng quốc tế page=29
Reading Hàng quốc tế page=30
Reading Hàng quốc tế page=31
Reading Hàng quốc tế page=32
Reading Hàng quốc tế page=33
Reading Hàng quốc tế page=34
Reading Hàng quốc tế page=35
Reading Hàng quốc tế page=36
Reading Hàng quốc tế page=37
Reading Hàng quốc tế page=38
Reading Hàng quốc tế page=39
Reading Hàng quốc tế page=40
Reading Hàng quốc tế page=41
Reading Hàng quốc tế page=42
Reading Hàng quốc tế page=43
Reading Hàng quốc tế page=44
Reading Hàng q

Reading Sách, VPP & Quà Tặng page=65
Reading Sách, VPP & Quà Tặng page=66
Reading Sách, VPP & Quà Tặng page=67
Reading Sách, VPP & Quà Tặng page=68
Reading Sách, VPP & Quà Tặng page=69
Reading Sách, VPP & Quà Tặng page=70
Reading Sách, VPP & Quà Tặng page=71
Reading Sách, VPP & Quà Tặng page=72
Reading Sách, VPP & Quà Tặng page=73
Reading Sách, VPP & Quà Tặng page=74
Reading Sách, VPP & Quà Tặng page=75
Reading Sách, VPP & Quà Tặng page=76
Reading Sách, VPP & Quà Tặng page=77
Reading Sách, VPP & Quà Tặng page=78
Reading Sách, VPP & Quà Tặng page=79
Reading Sách, VPP & Quà Tặng page=80
Reading Sách, VPP & Quà Tặng page=81
Reading Sách, VPP & Quà Tặng page=82
Reading Sách, VPP & Quà Tặng page=83
Reading Sách, VPP & Quà Tặng page=84
Reading Sách, VPP & Quà Tặng page=85
Reading Sách, VPP & Quà Tặng page=86
Reading Sách, VPP & Quà Tặng page=87
Reading Sách, VPP & Quà Tặng page=88
Reading Sách, VPP & Quà Tặng page=89
Reading Sách, VPP & Quà Tặng page=90
Reading Sách, VPP & Quà Tặng page=91
R

Reading Voucher - Dịch Vụ - Thẻ Cào page=64
Reading Voucher - Dịch Vụ - Thẻ Cào page=65
Reading Voucher - Dịch Vụ - Thẻ Cào page=66
Reading Voucher - Dịch Vụ - Thẻ Cào page=67
Reading Voucher - Dịch Vụ - Thẻ Cào page=68
Reading Voucher - Dịch Vụ - Thẻ Cào page=69
Reading Voucher - Dịch Vụ - Thẻ Cào page=70
Reading Voucher - Dịch Vụ - Thẻ Cào page=71
Reading Voucher - Dịch Vụ - Thẻ Cào page=72
Reading Voucher - Dịch Vụ - Thẻ Cào page=73
Reading Voucher - Dịch Vụ - Thẻ Cào page=74
Reading Voucher - Dịch Vụ - Thẻ Cào page=75
Reading Voucher - Dịch Vụ - Thẻ Cào page=76
Reading Voucher - Dịch Vụ - Thẻ Cào page=77
Reading Voucher - Dịch Vụ - Thẻ Cào page=78
Reading Voucher - Dịch Vụ - Thẻ Cào page=79
Reading Voucher - Dịch Vụ - Thẻ Cào page=80
Reading Voucher - Dịch Vụ - Thẻ Cào page=81
Reading Voucher - Dịch Vụ - Thẻ Cào page=82
Reading Voucher - Dịch Vụ - Thẻ Cào page=83
Reading Voucher - Dịch Vụ - Thẻ Cào page=84
Reading Voucher - Dịch Vụ - Thẻ Cào page=85
Reading Voucher - Dịch Vụ - Thẻ 

In [14]:
tablename = 'test'
query = f'SELECT * FROM {tablename} ORDER BY id DESC LIMIT 5'
cur.execute(query)
cur.fetchall()

[(142618,
  'https://salt.tikicdn.com/cache/200x200/ts/product/8a/76/d2/06d097fdf6da722b43a7dbb083788fd8.jpg',
  980000,
  'Voucher - Dịch Vụ - Thẻ Cào',
  'Voucher - Dịch vụ/Du lịch - Khách sạn/Tour du lịch/Tour trong nước/Tour miền Trung',
  'Tour Bà Nà, Buffet Trưa, Gồm Vé Cáp Treo, Đón Tận Nơi Tại Đà Nẵng',
  '',
  1150000,
  -15,
  95,
  4,
  0,
  'https://tiki.vn/tour-ba-na-buffet-trua-gom-ve-cap-treo-don-tan-noi-tai-da-nang-p6194587.html?src=category-page-11312&2hi=0'),
 (142617,
  'https://salt.tikicdn.com/cache/200x200/ts/product/bf/73/1c/473c7f25ee50b8e0c17aa2731a2877e7.jpg',
  171000,
  'Voucher - Dịch Vụ - Thẻ Cào',
  'Voucher - Dịch vụ/Du lịch - Khách sạn/Sim 3G/4G - Bộ phát Wifi',
  'Gohub - Sim 4G Thái Lan Truemove 3GB (GỌI + DATA + Public Wifi)',
  '',
  220000,
  -22,
  100,
  1,
  0,
  'https://tiki.vn/gohub-sim-4g-thai-lan-truemove-3gb-goi-data-public-wifi-p6498565.html?src=category-page-11312&2hi=0'),
 (142616,
  'https://salt.tikicdn.com/cache/200x200/ts/product/85

In [13]:
# j=0
# soup = load_website(cat[j][1],prefix='')
# articles = soup.find_all('div', {"class":'product-item'})
# links = soup.find_all('div',{"class":'list-pager'})
# soup = load_website(links[0].find_all('a',{"class":"normal"})[1]['href'],prefix='https://tiki.vn')
# articles = soup.find_all('div', {"class":'product-item'})
# k=36
# try:
#         images = articles[k].img['src']
#         fprice = int(articles[k].find_all("span",{"class":"final-price"})[0].text.strip().split()[0].strip('đ').replace('.',''))
#         rprice = [0 if articles[k].find_all("span",{"class":"price-regular"})[0].text == '' else int(articles[k].find_all("span",{"class":"price-regular"})[0].text.strip('đ').replace('.',''))][0]
#         discount = [0 if len(articles[k].find_all("span",{"class":"final-price"})[0].text.strip().split()) == 1 else int(articles[k].find_all("span",{"class":"final-price"})[0].text.strip().split()[1].split('%')[0])][0]
#         seller = articles[k]['data-brand']
#         titles = articles[k].a['title'].strip().replace('\'','').replace('"','')
#         subcategory = articles[k]['data-category'].strip()
#         category = cat[j][0]
# #         num_reviews = [int(articles[k].find_all('p',{"class":'review'})[0].text.strip('\(\)').split()[0]) if articles[k].find_all('p',{"class":'review'}) != [] or articles[k].find_all('p',{"class":'review'}) != 'Chưa có nhận xét' else 0][0]
#         ratings = [int(articles[k].find_all('span',{"class":'rating-content'})[0].find('span')['style'].split(':')[1].split('%')[0]) if articles[k].find_all('span',{"class":'rating-content'}) != [] else 0][0]
#         tikinow = [0 if articles[k].find_all('i',{"class":"tikicon icon-tikinow-20"}) == [] else 1][0]
#         productlink = articles[k].a['href']
# except Exception as err:
#     print(err)
    
# num_reviews = [int(articles[k].find_all('p',{"class":'review'})[0].text.strip('\(\)').split()[0]) if articles[k].find_all('p',{"class":'review'}) != [] or articles[k].find_all('p',{"class":'review'})[0].text != 'Chưa có nhận xét' else 0][0]
# print(articles[k].find_all('p',{"class":'review'})[0].text)
# [0 if articles[k].find_all('p',{"class":'review'}) == [] or articles[k].find_all('p',{"class":'review'})[0].text == 'Chưa có nhận xét' else int(articles[k].find_all('p',{"class":'review'})[0].text.strip('\(\)').split()[0])][0]

# if articles[k].find_all('p',{"class":'review'}) == [] or articles[k].find_all('p',{"class":'review'})[0].text == 'Chưa có nhận xét':
#     num_reviews = 0
# else:
#     num_reviews = int(articles[k].find_all('p',{"class":'review'})[0].text.strip('\(\)').split()[0])

# num_reviews
# print(articles[k].find_all('p',{"class":'review'})[0].text == 'Chưa có nhận xét')

In [29]:
soup = load_website('https://tiki.vn/may-dem-tien/c29006?src=tree','')

links = soup.find_all('div',{"class":'list-pager'})
        
articles = soup.find_all('div', {"class":'product-item'})
# #         print('Reading '+cat[j][1])
#         for k in range(len(articles)):
#             scrape_and_insert(cat, j, articles, k, cur, conn, tablename)
            
#         while links[0].find_all('a', {"class": "next"}) != []:
#             try:
#                 soup = load_website(links[0].find_all('a', {"class": "next"})[0]['href'],prefix='https://tiki.vn')
#                 articles = soup.find_all('div', {"class":"product-item"})
# #                 print('Reading',cat[j][0],links[0].find_all('a', {"class": "next"})[0]['href'].split('&')[1],sep=' ')
#                 links = soup.find_all('div',{"class":'list-pager'})
#                 for i in range(len(articles)):
#                     scrape_and_insert(cat, j, articles, i, cur, conn, tablename)
                
#             except:
#                 continue
                
articles[3]['data-id']

'15519162'